In [12]:
import numpy as np
import copy
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

import gym
from collections import deque
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16, 10)

In [13]:
class LinearLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear_layer = nn.Sequential(nn.Linear(in_features, out_features), nn.ReLU())
    def forward(self, x):
        x = self.linear_layer(x)
        return x

In [27]:
class Actor(nn.Module):
    def __init__(self, s_dim, a_dim, hidden_dim=256, num_hidden_layers=2):
        super(Actor, self).__init__()

        self.s_dim = s_dim
        self.a_dim = a_dim

        layers = [LinearLayer(s_dim, hidden_dim)]
        for _ in range(num_hidden_layers):
            layers.append(LinearLayer(hidden_dim, hidden_dim))
        layers.append(nn.Linear(hidden_dim, a_dim))

        self.f = nn.Sequential(*layers)

    def forward(self, state):
        if not isinstance(state, torch.Tensor):
            state = torch.tensor(state, dtype=torch.float32)
        out = self.f(state)
        #out = torch.tanh(out)
        out= F.softmax(out+1e-10, dim=-1)
        return out
    
    def sampling(self, state):

        probs = self(state)
        dist = Categorical(probs)
        sampled_actions = dist.sample()
        log_probs = dist.log_prob(sampled_actions).unsqueeze(1)
        return probs, log_probs


In [28]:
class QDNN(nn.Module):
    def __init__(self, s_dim, a_dim, hidden_dim=256, num_hidden_layers=2):
        super(QDNN, self).__init__()

        self.s_dim = s_dim
        self.a_dim = a_dim

        layers = [LinearLayer(s_dim + a_dim, hidden_dim)]
        for _ in range(num_hidden_layers):
            layers.append(LinearLayer(hidden_dim, hidden_dim))
        layers.append(nn.Linear(hidden_dim, 1))

        self.f = nn.Sequential(*layers)

    def forward(self, state, action):
        x = torch.cat([state, action], dim=1)
        out = self.f(x)
        return out

In [29]:
class Critic(nn.Module):
    def __init__(self, s_dim, a_dim, hidden_dim=256, num_hidden_layers=2):
        super(Critic, self).__init__()

        self.s_dim = s_dim
        self.a_dim = a_dim

        # Two DNNs to mitigate positive bias
        self.Q1 = QDNN(s_dim, a_dim, hidden_dim, num_hidden_layers)
        self.Q2 = QDNN(s_dim, a_dim, hidden_dim, num_hidden_layers)

    def forward(self, state, action):
        q1 = self.Q1(state, action)
        q2 = self.Q2(state, action)
        return q1, q2

In [42]:
class SAC():
    def __init__(self, s_dim, a_dim, hidden_dim_actor=256, hidden_dim_critic=256, 
                 num_layer_actor=2, num_layer_critic=2, lr_act=3e-4, lr_crit=3e-4, 
                 gamma=0.99, tau=0.005, alpha=0.2, lambd=0.005, target_upd_inter=1, 
                 buffer_capacity=1000, batch_size=32, grad_steps = 1, device="cpu"):
        
        self.s_dim = s_dim
        self.a_dim = a_dim
        self.device = device
        self.batch_size = batch_size
        self.buffer = deque(maxlen=buffer_capacity)
        self.grad_steps = grad_steps

        self.alpha = alpha
        self.lambd = lambd
        self.gamma = gamma
        self.tau = tau
        self.target_upd_inter = target_upd_inter

        self.actor = Actor(s_dim, a_dim, hidden_dim_actor, num_hidden_layers=num_layer_actor).to(device)
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=lr_act)

        self.critic = Critic(s_dim, a_dim, hidden_dim_critic, num_hidden_layers=num_layer_critic).to(device)
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=lr_crit)

        # Stabilize training
        self.critic_target = copy.deepcopy(self.critic).to(device)

    def choose_action(self, state):
        self.actor.eval()
        if not isinstance(state, torch.Tensor):
            state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        with torch.no_grad():
            action_probs = self.actor(state.to(self.device))
            dist = Categorical(action_probs)
            action = dist.sample()
        return action.item(), action_probs.squeeze().detach().cpu().numpy()

    def train(self, update_interval):
        if len(self.buffer) < self.batch_size:
            return None, None

        for i in range(self.grad_steps):
            batch = random.sample(self.buffer, self.batch_size)
            states, actions, rewards, next_states, dones = zip(*batch)
            states = torch.tensor(states, dtype=torch.float32).to(self.device)
            actions = torch.tensor(actions, dtype=torch.float32).squeeze().to(self.device)
            rewards = torch.tensor(rewards, dtype=torch.float32).unsqueeze(1).to(self.device)
            next_states = torch.tensor(next_states, dtype=torch.float32).to(self.device)
            dones = torch.tensor(dones, dtype=torch.float32).to(self.device)
            #print(dones)
            
            # # Critic train
            # self.critic.train()
            
            # with torch.no_grad():
            #     probs = self.actor(next_states)
            #     dist = Categorical(probs)
            #     sampled_actions = dist.sample()
            #     next_actions_log_probs = dist.log_prob(sampled_actions).unsqueeze(1)
            #     q1_next, q2_next = self.critic_target(next_states, probs)
            #     min_q_next = torch.min(q1_next, q2_next)
            #     #print((torch.ones_like(dones)-dones).shape)
            #     target_q_value = rewards + self.gamma *(torch.ones_like(dones)-dones).unsqueeze(1)*(min_q_next - self.alpha * next_actions_log_probs)#.sum(-1, keepdim=True))
                
            # q1, q2 = self.critic(states, actions)
            
            # critic_loss = F.mse_loss(q1, target_q_value.detach()) + F.mse_loss(q2, target_q_value.detach())

            # self.critic_optimizer.zero_grad()
            # critic_loss.backward()
            # self.critic_optimizer.step()

            # # Actor train
            # self.actor.train()

            # probs = self.actor(states)
            # dist = Categorical(probs)
            # sampled_actions = dist.sample()
            # log_probs = dist.log_prob(sampled_actions)#.sum(-1, keepdim=True)
            # q1_actor, q2_actor = self.critic(states, probs)
            # min_q_actor = torch.min(q1_actor, q2_actor)
            
            # actor_loss = (self.alpha * log_probs.unsqueeze(1) - min_q_actor).mean()

            # self.actor_optimizer.zero_grad()
            # actor_loss.backward()
            # self.actor_optimizer.step()

            critic_loss = self.critic_train(states, actions, next_states, rewards, dones)

            # Actor train
            actor_loss = self.train_actor(states)

            # Soft update of target networks
            if update_interval % self.target_upd_inter == 0:
                for target_parameters, parameters in zip(self.critic_target.parameters(), self.critic.parameters()):
                    target_parameters.data.copy_(self.tau * parameters.data + (1.0 - self.tau) * target_parameters.data)

        #print(critic_loss.item(), actor_loss.item())
        return critic_loss, actor_loss
    
    def critic_train(self, states, actions, next_states, rewards, dones):
        with torch.no_grad():
            probs, actions_log_probs = self.actor.sampling(states)
            q1_next, q2_next = self.critic_target(next_states, probs)
            min_q_next = torch.min(q1_next, q2_next)
            #print((torch.ones_like(dones)-dones).shape)
            target_q_value = rewards + self.gamma *(torch.ones_like(dones)-dones).unsqueeze(1)*(min_q_next - self.alpha * actions_log_probs)#.sum(-1, keepdim=True))
                
        q1, q2 = self.critic(states, actions)
        
        critic_loss = F.mse_loss(q1, target_q_value.detach()) + F.mse_loss(q2, target_q_value.detach())

        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        return critic_loss.item()
    
    def train_actor(self, states):
        probs, log_probs = self.actor.sampling(states)
        q1_actor, q2_actor = self.critic(states, probs)
        min_q_actor = torch.min(q1_actor, q2_actor)
        
        actor_loss = (self.alpha * log_probs.unsqueeze(1) - min_q_actor).mean()

        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        return actor_loss.item()

    def add_elements_to_buffer(self, state, probs, reward, next_state, done):
        self.buffer.append((state, probs, reward, next_state, done))

In [43]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [44]:
### Plot the learning progress
def plot_scores(score_history):
    # Create the plot
    fig = plt.figure(figsize=(20, 6))
    ax = fig.add_subplot(111)

    # Plot the scores with specified colors and labels
    ax.plot(np.arange(1, len(score_history) + 1), score_history, color='green', label='SAC')

    # Set the labels with a larger font size
    ax.set_ylabel('Total reward (= time balanced)', fontsize=20)
    ax.set_xlabel('Episode #', fontsize=20)

    # Set the tick labels to a larger font size
    ax.tick_params(axis='both', which='major', labelsize=15)

    # Add a legend with a specified font size
    ax.legend(fontsize=20)

    # Show the plot
    plt.show()

In [45]:
def train_SAC(agent, env, n_games, training_interval=1, update_interval=1, print_num_episodes=10):
    env.seed(0)
    n_games = n_games

    best_score = env.reward_range[0]
    score_history = []

    avg_score = 0
    n_steps = 0
    training_interval=training_interval
    update_interval = update_interval


    for i in range(n_games):
        observation = env.reset()
        done = False
        score = 0
        while not done:
            action, probs = agent.choose_action(observation)
            observation_, reward, done, _ = env.step(action)
            score += reward
            agent.add_elements_to_buffer(observation, probs, reward, observation_, done)
            observation = observation_
            update_interval += 1
            n_steps += 1
            if n_steps % training_interval == 0:
                critic_loss, actor_loss = agent.train(update_interval)

        score_history.append(score)
        avg_score = np.mean(score_history[-100:])
        if i%print_num_episodes == 0:
            print('episode', i, 'score %.1f' % score, 'avg score %.1f' % avg_score)
            #print(critic_loss, actor_loss, value_loss)
    
    env.close()

    return score_history

## Cartpole

In [47]:
env = gym.make("CartPole-v0")

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
agent = SAC(state_dim, action_dim, buffer_capacity=int(1e6), hidden_dim_actor = 64, hidden_dim_critic=64, 
            num_layer_actor=2, num_layer_critic=2, lr_act=6e-4, lr_crit=6e-4, alpha=0.2, tau=0.05, batch_size=20, 
            grad_steps=2, device=device)

score_history = train_SAC(agent, env, 200, training_interval=1, update_interval=1, print_num_episodes=5)
plot_scores(score_history)

c:\Users\guill\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\envs\registration.py:592: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
c:\Users\guill\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\guill\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\guill\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\core.py:268: DeprecationWar

episode 0 score 19.0 avg score 19.0
episode 5 score 10.0 avg score 16.7
episode 10 score 10.0 avg score 13.6
episode 15 score 8.0 avg score 12.2
episode 20 score 9.0 avg score 12.6
episode 25 score 9.0 avg score 11.8
episode 30 score 8.0 avg score 11.3
episode 35 score 33.0 avg score 15.4
episode 40 score 64.0 avg score 20.0
episode 45 score 59.0 avg score 23.0
episode 50 score 200.0 avg score 29.9
episode 55 score 200.0 avg score 44.0
episode 60 score 200.0 avg score 56.5
episode 65 score 200.0 avg score 66.7
episode 70 score 200.0 avg score 75.3


KeyboardInterrupt: 

## Cartpole-v1

In [52]:
env = gym.make("CartPole-v1")

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
agent = SAC(state_dim, action_dim, buffer_capacity=int(1e6), hidden_dim_actor = 64, hidden_dim_critic=64, 
            num_layer_actor=1, num_layer_critic=1, lr_act=6e-4, lr_crit=6e-4, alpha=0.2, tau=0.05, batch_size=32, 
            grad_steps=2, device=device)

score_history = train_SAC(agent, env, 250, training_interval=1, update_interval=1, print_num_episodes=5)
plot_scores(score_history)

episode 0 score 31.0 avg score 31.0
episode 5 score 9.0 avg score 16.7
episode 10 score 9.0 avg score 13.2
episode 15 score 10.0 avg score 12.1
episode 20 score 10.0 avg score 13.0
episode 25 score 46.0 avg score 15.8
episode 30 score 123.0 avg score 26.9
episode 35 score 325.0 avg score 51.5
episode 40 score 276.0 avg score 94.6
episode 45 score 497.0 avg score 130.5
episode 50 score 380.0 avg score 160.1
episode 55 score 292.0 avg score 179.6
episode 60 score 500.0 avg score 205.0
episode 65 score 500.0 avg score 226.4
episode 70 score 500.0 avg score 245.7
episode 75 score 500.0 avg score 260.5
episode 80 score 500.0 avg score 275.3
episode 85 score 500.0 avg score 288.3
episode 90 score 500.0 avg score 300.0
episode 95 score 500.0 avg score 310.4
episode 100 score 500.0 avg score 320.3
episode 105 score 500.0 avg score 344.6
episode 110 score 500.0 avg score 369.2


KeyboardInterrupt: 